<a href="https://colab.research.google.com/github/PiyumaliSandunika/e18-4yp-Multimodal-Emotion-Prediction-Using-Reinforcement-Learning/blob/main/online_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


** Model Loading**

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch

# Load the saved model
model_path = "/content/drive/MyDrive/FYP_Text/"
loaded_model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


In [ ]:
def predict_emotion_with_probabilities(sentence, model, tokenizer):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors="pt")

    # Perform inference
    outputs = model(**inputs)
    logits = outputs.logits

    # Apply softmax to get probabilities
    probabilities = torch.nn.functional.softmax(logits, dim=1)

    # Convert probabilities tensor to list
    probabilities_list = probabilities.squeeze().tolist()

    # Map predicted label to emotion
    emotion_labels = ["happiness", "sadness", "anger", "neutral"]
    predicted_emotion = emotion_labels[torch.argmax(logits, dim=1).item()]

    return probabilities_list, predicted_emotion


In [ ]:

# Example sentences for prediction
sentences = [
    # "I am a broken-hearted",
    # "I am emotionaly damaged right now",
    # "Do what makes feel you better",
    # "I feel nervous",
    # "I am a student",
    # "You're stupid and it makes me anger",
    # "Go to hell",
    # "You are the worst,I do not want to see you",
    # "I am angry right now",
    # "You are my sunshine",
    # "Such a wonderful performance",
    # "I am lonely",
    # "This is strange! anyways bravo to myself!",
    # "Things are getting stranger",
    # "I lumped my throat",
    # "Billy, your face is like a wet weekend",
    "I’m on cloud nine"
]

# Perform prediction for each sentence
for sentence in sentences:
    predicted_emotion = predict_emotion_with_probabilities(sentence,loaded_model,tokenizer)
    print(f"Sentence: '{sentence}'")
    print(f"Predicted Emotion: {predicted_emotion}\n")

Sentence: 'I’m on cloud nine'
Predicted Emotion: ([2.455699586789706e-06, 2.885532694563153e-06, 1.1664121757348767e-06, 0.9999934434890747], 'neutral')



In [ ]:
def active_learning_selection(user_data, model, threshold=0.5):
    selected_examples = []
    for text, label in user_data:
        encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        outputs = model(**encoded_input)
        probabilities = torch.softmax(outputs.logits, dim=1)
        confidence = probabilities.max().item()
        if confidence < threshold:
            selected_examples.append((text, label))
    return selected_examples

# Function for online learning update
def online_learning_update(new_data, model, optimizer, loss_fn):
    for text, label in new_data:
        encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        outputs = model(**encoded_input, labels=torch.tensor([label]))
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return model
def is_confident(predicted_label, confidence_threshold=0.7):
    # Check if the prediction confidence is above the threshold
    return confidence_threshold <= torch.softmax(predicted_label, dim=1).max()

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

# Initialize DistilBERT tokenizer and load the fine-tuned model
model_path = "/content/drive/MyDrive/FYP_Text/"
loaded_model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Define label mappings
label_map = {
    0: "happiness",
    1: "sadness",
    2: "anger",
    3: "neutral"
}

# Function for online learning update
def online_learning_update(new_data, model, optimizer, loss_fn):
    for text, label in new_data:
        encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        encoded_input['labels'] = torch.tensor([label])
        outputs = model(**encoded_input)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return model

# Main loop for user interaction
user_data = []

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)  # Optimizer for online learning
loss_fn = torch.nn.CrossEntropyLoss()  # Loss function for online learning

while True:
    # Receive input from the user
    user_input = input("Enter text: ")

    # Perform inference with the current model
    encoded_input = tokenizer(user_input, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**encoded_input)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    print(f"Predicted emotion: {label_map[predicted_label]}")

    # Ask for user feedback
    user_label = int(input("Enter correct label (happiness:0, sadness:1, anger:2, neutral:3): "))
    user_data.append((user_input, user_label))

    # Retrain the model with the new data
    model = online_learning_update([(user_input, user_label)], model, optimizer, loss_fn)

    # Perform inference with the updated model
    outputs = model(**encoded_input)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    print(f"New predicted emotion: {label_map[predicted_label]}")

    # Prompt the user for more input
    more_data = input("Do you have more data to provide? (yes/no): ")
    if more_data.lower() != "yes":
        break


Enter text: I’m on cloud nine
Predicted emotion: neutral
Enter correct label (happiness:0, sadness:1, anger:2, neutral:3): 0
New predicted emotion: happiness
Do you have more data to provide? (yes/no): no


In [ ]:
# Perform inference with the updated model
encoded_input = tokenizer("I'm on cloud", padding=True, truncation=True, return_tensors='pt')
outputs = model(**encoded_input)
predicted_label = torch.argmax(outputs.logits, dim=1).item()
print(f"New predicted emotion: {label_map[predicted_label]}")
